## Create Operations Rehearsal Blocks from and Opsim data base

Take a OpsSim database created by the simulated scheduler for the 3rd operations rehearsal and produce blocks for the for the scheduler CSC.

CWW - March 2024

In [9]:
%config InlineBackend.figure_format = 'retina'

from lsst.ts.observing import ObservingBlock, ObservingScript

from sqlalchemy import create_engine

import pandas as pd

import matplotlib
import matplotlib.pyplot as pyplot

Let's get the data base into a Pandas DataFrame.  We will use sqlalchemy to make a database connection and then pandas to read the database tables into DataFrames.  This could have SQL selections in it but I am just reading the whole table in.

In [ ]:
engine = create_engine('sqlite:////sdf/data/rubin/shared/ops-rehearsal-3/scheduler_sims/ops_rehearsal_apr_2024_v3_playbackBlock.db')
Observations = pd.read_sql_table('observations', engine)
Observations.head()

In [ ]:
# Summarize and then select columns we want to use

print(Observations.columns)

# Make a df with selected values.

visit_info = Observations[['observationId', 'fieldRA', 'fieldDec', 
    'observationStartMJD',
    'filter', 'rotSkyPos', 'rotTelPos', 
    'visitTime', 'visitExposureTime',
    'altitude', 'azimuth', 'slewTime',
    'note', 'target', 'fieldId',
    'block_id', 'playbackBlock']]

In [ ]:
#Print out the BLOCK-IDs.
pd.unique(visit_info.playbackBlock)

In [ ]:
#Group the data frame by block and print how many are in each block.
BLOCKS = visit_info.groupby(['playbackBlock'])
BLOCKS.observationId.count()

### Extract the data frames we will use for the blocks.

In [ ]:
#Get BLOCK = 0
block = BLOCKS.get_group( (0,) )

In [ ]:
# Create two extracted data frames with the correct field names.  We will 
# iterate over them later.  Also add some constant values to the data frame.
# one is for the sleep command (for the slew), one for the take_image command.

slew_block = block[['slewTime']]
slew_block.columns = ['sleep_for']

obs_block = block[['fieldRA', 'fieldDec', 'visitExposureTime', 'target', 'filter']]
obs_block.columns = ['ra', 'dec', 'exp_times', 'note', 'filter']

obs_block.insert(len(obs_block.columns), 'image_type', 'OBJECT')
obs_block.insert(len(obs_block.columns), 'nimages', 1)

# Print out the first five rows of the obs and sleep data frames
display(obs_block.head())
display(slew_block.head())

### Produce the scripts and blocks

In [ ]:
# Make a function to print the json structure in a multi-line human readable 
# form for the blocks GitHub Repository.

def pretty_print_model(model):
    print(model.model_dump_json(indent=4))

In [ ]:
# Set how many rows of the BLOCK to process
# rows = obs_block.shape[0]
rows = 3

# All the scripts in the block will be appended to this list.
script_list = []

# First write the script that puts us into playlist mode.
playlist_script = ObservingScript(name="run_command.py",  standard=True,
            parameters={"component": "CCCamera", "cmd": "play", 
                        "parameters": {"playlist": 'ops3_60', "repeat": False}})

script_list.append(playlist_script)

for row in range(rows):

    # Convert this row of the data frames parameters to dictionaries.
    slew_dict = slew_block.iloc[[row],:].to_dict('records')[0]
    obs_dict = obs_block.iloc[[row],:].to_dict('records')[0]

    # Place the the sleep parameters into a script
    sleep_script = ObservingScript(name="sleep.py",  standard=True,
            parameters= slew_dict)
    
    # Place the the ComCam image parameters into a script
    take_image_script = ObservingScript(name="take_image_comcam.py", standard=True, 
                                        parameters = obs_dict)

    # Append them to the running script list    
    script_list.append(sleep_script)
    script_list.append(take_image_script)

# Build the block dictionary
observing_block = ObservingBlock(name="BLOCK-XXX", program="BLOCK-XXX", scripts=script_list) 

pretty_print_model(observing_block)